In [1]:
with open('/Users/richardli/Desktop/openani/malicious_filtering_for_ani/data/external_raw/dictionaries/FIFA.txt', 'r') as file:
    print(file.read()[:200])

﻿阿巴多纳多
阿巴尔多纳多
阿巴罗斯
阿巴特
阿邦丹契里
阿邦丹切里
阿邦丹谢里
阿比达尔
阿比尔德加德
阿比亚蒂
阿伯丁
阿伯里斯特威斯城
阿部博之
阿布德萨基
阿卜杜尔卡迪尔卡亚利
阿布杜拉曼
阿卜杜拉耶
阿卜杜特劳雷
阿布多恩
阿布多拉耶迪亚拉
阿布法拉格
阿布凯塔
阿布拉哈姆森
阿布拉罕
阿布拉罕冈萨雷斯
阿布蓝特斯
阿布里埃尔
阿布鲁斯卡托
阿布罗斯
阿达尔托
阿达莫库利巴利
阿达莫维奇


In [2]:
import sys

sys.path.append('/Users/richardli/Desktop/openani/malicious_filtering_for_ani/malicious_filtering_for_ani')

In [4]:
from malicious_filtering_for_ani.utils.jieba_util import init_jieba

init_jieba("/Users/richardli/Desktop/openani/malicious_filtering_for_ani/configs/path_config.yaml")


Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/95/g5q43jrj22x4zqjq90qtpncr0000gn/T/jieba.cache
Loading model cost 0.253 seconds.
Prefix dict has been built successfully.


In [ ]:
import pandas as pd
import re


def extract_text_column(input_file, output_file):
    # Load the CSV file into a DataFrame
    df = pd.read_csv(input_file, delimiter=',')

    # Extract the 'TEXT' column
    text_column = df["review"]

    # Save the column to a new file
    text_column.to_csv(output_file, index=False, header=False)


def clean_text_line(line):
    """
    Cleans a single line by removing spaces between words and retaining spaces between numbers and words.
    """
    # Step 1: Remove spaces between words
    text_without_word_spaces = re.sub(r'([^\d\s])\s+([^\d\s])', r'\1\2', line)

    # Step 2: Ensure spaces are kept between numbers and words
    text_with_number_word_spaces = re.sub(r'(\d)\s+([^\d])', r'\1 \2', text_without_word_spaces)
    text_with_number_word_spaces = re.sub(r'([^\d])\s+(\d)', r'\1 \2', text_with_number_word_spaces)

    return text_with_number_word_spaces


def process_text_file(input_file, output_file):
    """
    Reads text from the input file line by line, applies cleaning transformations,
    and saves the modified text to the output file.

    Args:
    - input_file (str): Path to the input text file.
    - output_file (str): Path to the output text file.
    """
    with open(input_file, 'r', encoding='utf-8') as f_in, open(output_file, 'w', encoding='utf-8') as f_out:
        # Read and process each line
        for line in f_in:
            cleaned_line = clean_text_line(line)

            # Write the cleaned line to the output file, including the newline character
            f_out.write(cleaned_line)

    print(f"Processed text saved to {output_file}")

In [ ]:
import requests
import simplejson as json


def download_and_read_csv(url):
    """
    Downloads a CSV file from the given URL and reads it into a pandas DataFrame.

    Args:
    - url (str): The URL of the CSV file.

    Returns:
    - pd.DataFrame: A DataFrame containing the content of the CSV file.
    """
    # Download the content from the URL
    response = requests.get(url)
    response.raise_for_status()  # Raise an error if the download fails

    # Convert the content to a string
    csv_content = response.content.decode('utf-8')

    # Read the CSV content into a DataFrame
    from io import StringIO
    df = pd.read_csv(StringIO(csv_content))

    df.to_csv("hotel_all.csv", index=False)

In [ ]:
url = 'https://cas.dandanplay.net/api/comment/175690008?from=0&related=b-ep713744-1,gm-32644&urls=https%3A%2F%2Fwww.bilibili.com%2Fbangumi%2Fplay%2Fep713744%3Ffrom_spmid%3D666.25.episode.0%7Chttps%3A%2F%2Fani.gamer.com.tw%2FanimeVideo.php%3Fsn%3D32644&layer=2&chConvert=0'

# Make a GET request to fetch the external_raw JSON content
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON content
    json_data = response.json()

    # Save the JSON data to a file
    with open('dandanplaytest.json', 'w', encoding='utf-8') as json_file:
        json.dump(json_data, json_file, indent=4)
    print("JSON data has been downloaded and saved to data.json")
else:
    print(f"Failed to download JSON data. HTTP Status code: {response.status_code}")